# Optik Karakter Tanıma Uygulaması

### Geliştirme ortamına atanan ekran kartının kontrolü

* Geliştirmeye başlamadan önce geliştirme ortamında GPU kullanımının aktifleştirildiğinden emin olunması için yukarıda "Runtime" sekmesinden "Change runtime type"a tıklayalım. Eğer GPU seçeneği seçili değilse seçim listesinden GPUyu seçelim. Bu durumda bu Python Notebook'u tekrardan yüklememiz gerekebilir.
* GPU'nun aktif olup olmadığını ve bize atanan GPU'nun özelliklerini aşağıdaki komutu çalıştırarak görebiliriz.

In [ ]:
!nvidia-smi

## Colab ile Google Drive bağlantısının kurulması

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

## Google Drive'da çalışılacak klasörün oluşturulması ve klasörün konumuna geçiş

In [ ]:
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima
!mkdir OCR
%cd OCR

## Github'dan Metin Tespiti ve Metin Tanıma repository'lerin yüklenmesi

In [ ]:
# Metin Tespiti repo'sunun kopyalanması
!git clone https://github.com/SakuraRiven/EAST.git
# Kodun bulunduğu konuma giriş
%cd EAST
# Modellerin koyulacağı klasörün oluşturulması
!mkdir pths
# Model klasörüne giriş
%cd pths
# Önceden eğitilmiş modellerin indirilmesi
file_download_link = 'https://drive.google.com/uc?export=download&id=1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg'
!wget -O east_vgg16.pth --no-check-certificate "$file_download_link"
# Kodun bulunduğu klasöre geri dönüş
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR

!rm -r deep_text_recognition_benchmark
# Metin Tanıma repo'sunun kopyalanması
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git
# Kodun bulunduğu konuma giriş
%cd deep-text-recognition-benchmark
# Modellerin koyulacağı klasörün oluşturulması
!mkdir pths
# Model klasörüne giriş
%cd pths
# Önceden eğitilmiş modellerin indirilmesi
# !gdown https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
!gdown https://drive.google.com/uc?id=1ajONZOgiG9pEYsQ-eBmgkVbMDuHgPCaY
# Kodun bulunduğu klasöre geri dönüş
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR
# '-', klasör ismini okurken sorun çıkarabildiğinden '_' karakteriyle değiştirelim
!mv deep-text-recognition-benchmark deep_text_recognition_benchmark

In [ ]:
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR
!ls

## Verilerin Hazırlanması, Okunması; Modellerin Hazırlanması ve Eğitimi

In [ ]:
!ls

Kullanılacak kütüphaneleri yüklemeden önce /content/gdrive/My Drive/OCR/EAST/detect.py dosyasında iki küçük değişiklik yapmamız gerekiyor.
* 4. satır "from EAST.model import EAST" koduyla değiştirilmeli
* 6. satır "from EAST.dataset import get_rotate_mat" koduyla değiştirilmeli

In [ ]:
# kullanılacak kütüphanelerin ve paketlerin ortama eklenmesi
!pip install lanms-proper
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR
import os
import time
import string
import shutil
import subprocess

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
import torch.optim as optim
from torch.utils import data
from torch.optim import lr_scheduler

import EAST.detect as detect
from EAST.model import EAST as EASTModel
from EAST.loss import Loss
from EAST.dataset import custom_dataset
from deep_text_recognition_benchmark.dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
from deep_text_recognition_benchmark.dataset import ResizeNormalize

%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 12.0) # görselin boyutları
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

### Verinin Yüklenmesi ve Hazırlanması

Üzerinde çalışacağımız veri, ICDAR 2015 eğik metin verisidir. Bu verinin indirilmesi https://rrc.cvc.uab.es/?ch=4&com=downloads adresinden yapılmalıdır.
Öncelikle sisteme e-mail adresinizle kaydolmanız gerekiyor. Sonrasında yine bu adresten "Task 4.4: End to End (2015 edition)" kısmına gelip;

* Training Set Images (88.5MB).
* Training Set Localisation and Transcription Ground Truth (157KB).
* Test Set Images (43.3MB).
* Test Set Ground Truth (244Kb).

Öğelerini tıklayarak indirmeniz gerekiyor.
Sonrasında;
* Training Set Images içeriğini, train_img klasörüne,
* Trainin Set Localisation and Transcription Ground Truth içeriğini train_gt klasörüne,
* Test Set Images içeriğini, test_img klasörüne,
* Test Set Ground Truth içeriğini ise, test_gt klasörüne çıkarıp,
Oluşturduğunuz bu klasörüleri ICDAR_2015 adında bir klasöre koymanız gerekiyor.

ICDAR_2015 klasörünü ise, drive'da oluşturmuş olduğunuz OCR klasörünün içine yüklemeniz gerekiyor.
Drive'a yükleme kısmı biraz zaman alabileceğinden, burada, yanınıza bir çay ya da kahve almak için ara vermenin zamanı olabilir :)

Yukarıdaki işlemleri yaptıktan sonra, size bu ders için sağlamış olduğumuz metinleri_cikar.py dosyasını da yine drive'daki OCR klasörüne yüklemelisiniz.
Tüm bu işlemlerin sonunda, drive'daki OCR klasörünün içinde:
* deep-text-recognition-benchmarl
* EAST
* ICDAR_2015
* metinleri_cikar.py
dosya ve klasörlerinin olması gerekiyor.

In [ ]:
# verilerin ve kodun doğru yüklendiğini doğrulayalım
!ls

#### Metin Tanıma Verisinin Hazırlanması

Metin Tanıma için hazırlanmış kod, lmdb databes isimli bir format kullandığından, veriden bu formatın hazırlanmasını sağlayalım
Bu script, ICDAR_2015 verisi içindeki resimlerden, metinlerin bulunduğu kısımları keserek yine drive'da OCR'ın içerisinde ICDAR_2015_metinler isimli yeni bir klasöre aktaracak. Bu klasör lmdb database'in hazırlanması için kullanılacak.

In [ ]:
# metin resimlerinin kesilip kaydedilmesi
!python metinleri_cikar.py
!ls
# NOT: videoda test_img için metinleri çıkarma kısmının sizde olmayacağından
# bahsedilmişti, fakat en altta performans ölçümünde kullanılacağından yeniden
# dahil edildi

In [ ]:
!ls

In [ ]:
# lmdb dataset'lerin oluşturulması
!pip install fire
# bu kısım, lmdb dataseti tekrar oluşturmak istediğinizde takılma olursa; önceden
# oluşturduğunuz metinler klasörünü silip üstteki hücreyi tekrar çalıştırarak
# oluşturduğunuzda buradaki kod da yine sorunsuz çalışmalıdır.
if not os.path.isdir('lmdb_train'):
    !python deep_text_recognition_benchmark/create_lmdb_dataset.py --inputPath ICDAR_2015_metinler/ --gtFile ICDAR_2015_metinler/train_img_gt.txt --outputPath lmdb_train/

### Metin Tespiti

#### Metin Tespiti Model Eğitimi için Verinin Okunması

In [ ]:
# Metin Tespiti modelinin eğitimi için verinin okunmaya hazır hale getirilmesi

# verinin konumu
train_img_path = os.path.abspath('./ICDAR_2015/train_img')
train_gt_path  = os.path.abspath('./ICDAR_2015/train_gt')
# modelin konumu
pths_path = 'EAST/pths'

In [ ]:
detection_batch_size = 8  # toplu resim sayısı
num_workers    = 4  # paralelde çalışacak veri hazırlayıcı thread sayısı
file_num = len(os.listdir(train_img_path))  # dosya sayısı
detection_trainset = custom_dataset(train_img_path, train_gt_path)
detection_train_loader = data.DataLoader(detection_trainset, batch_size=detection_batch_size, \
                               shuffle=True, num_workers=num_workers, drop_last=True)

#### Metin Tespiti Eğitim Kodunun Hazırlanması ve Modelin Eğitimi

In [ ]:
criterion = Loss()
device = torch.device("cuda:0")
model = EASTModel(pretrained=False).to(device)
# önceden eğitilmiş olan model üzerine fine-tuning yapacaksak
# model.load_state_dict(torch.load(model_dosyası))

# öğrenme oranı (learning rate)
lr             = 1e-3
# toplam epoch sayısı
epoch_iter     = 10  # 600 ile eğitim iyi bir sonuc verir
# model kaydetme sıklığı
save_interval  = 5

# modelin optimizasyonu için Adam optimizasyon yöntemi kullanılacak
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# Eğitimin yarısında öğrenme oranı (learning rate), 10'da 1'ine düşürülecek
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[epoch_iter//2], gamma=0.1)

In [ ]:
# Modelin Eğitilmesi
torch.cuda.empty_cache()
model.train()
for epoch in range(epoch_iter):	
    epoch_loss = 0
    epoch_time = time.time()
    for i, (img, gt_score, gt_geo, ignored_map) in enumerate(detection_train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)
        
        epoch_loss += loss.item()
        optimizer.zero_grad()  # gradient'leri sıfırlama
        loss.backward()  # back-propagation
        optimizer.step()
        scheduler.step()  # eğitimin yarısına gelindiğinde bu kod lr'yi 1e-4 yapacak

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
            epoch+1, epoch_iter, i+1, int(file_num/detection_batch_size), time.time()-start_time, loss.item()))
    
    print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/detection_batch_size), time.time()-epoch_time))
    print(time.asctime(time.localtime(time.time())))
    print('='*50)
    # modelin kaydedilmesi
    if (epoch + 1) % save_interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, os.path.join(pths_path, 'model_epoch_{}.pth'.format(epoch+1)))

In [ ]:
!ls /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR/EAST/pths

### Metin Tanıma

#### Metin Tanıma Ayarları

In [ ]:
# Metin Tanıma Model Ayarları
# tüm argümanların detayları deep_text_recognition_benchmark/train.py'da bulunabilir
import string
class ModelParametreleri:

    def __init__(self):
        # eğitim ayarları
        self.num_iter = 3000
        self.valInterval = 500
        self.continue_model = '/content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR/deep_text_recognition_benchmark/pths/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth'
        self.optimizer = 'Adadelta'
        self.lr = 1
        self.beta1 = 0.9
        self.rho = 0.95
        self.eps = 1e-8
        self.grad_clip = 5
        self.num_gpu = 1
        self.exp_name = 'training'

        # veri ayarları
        self.train_data = './lmdb_train'
        self.valid_data = './lmdb_test'
        self.batch_size = 192
        self.workers = 4
        self.select_data = '/'
        self.PAD = False
        self.batch_ratio = '1'
        self.total_data_usage_ratio = 1.0
        self.data_filtering_off = False
        self.rgb = False
        self.sensitive = True  # küçük/büyük karakter için eğitilsin mi

        # model ayarları
        self.Transformation = 'TPS'
        self.FeatureExtraction = 'ResNet'
        self.SequenceModeling = 'BiLSTM'
        self.Prediction = 'Attn'
        self.character = '0123456789abcdefghijklmnopqrstuvwxyz'
        self.num_fiducial = 20
        self.output_channel = 512
        self.hidden_size = 256
        self.imgH = 32  # modele verilecek resmin boyu
        self.imgW = 100  # modele verilecek resmin eni
        self.input_channel = 1  # model siyah beyaz resimler üzerinde çalışıyor
        self.batch_max_length = 25
        self.select_data = self.select_data.split('-')
        self.batch_ratio = self.batch_ratio.split('-')

        if self.sensitive:
            self.character = string.printable[:-6]  # same with ASTER setting (use 94 char).
        self.num_class = len(self.character) + 2  # buradaki +2, tahmine başlangıç ve bitiş iki özel karakteri için ekleniyor
opt = ModelParametreleri()

#### Metin Tanıma Model Eğitimi için Verinin Okunması

In [ ]:
# Metin Tanıma modelinin eğitimi için verinin okunmaya hazır hale getirilmesi
os.makedirs('saved_models/training', exist_ok=True)
# AlignCollate, farklı resimleri bir 'batch' haline getirir
# AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)

train_dataset = Batch_Balanced_Dataset(opt)

#### Metin Tanıma Eğitim Kodunun Hazırlanması ve Modelin Eğitimi

In [ ]:
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR/deep_text_recognition_benchmark/
# modelin oluşturulması
from collections import OrderedDict
from utils import AttnLabelConverter, Averager
from model import Model

converter = AttnLabelConverter(opt.character)
opt.num_class = len(converter.character)
model = Model(opt)

print('model input parameters', opt.imgH, opt.imgW, opt.num_fiducial, opt.input_channel, opt.output_channel,
    opt.hidden_size, opt.num_class, opt.batch_max_length, opt.Transformation, opt.FeatureExtraction,
    opt.SequenceModeling, opt.Prediction)

if opt.continue_model != '':
    # önceden eğitilmiş modelin yüklenmesini sağlayan fonksiyon
    def copy_state_dict(model, state_dict):
        if list(state_dict.keys())[0].startswith("module"):
            start_idx = 1
        else:
            start_idx = 0
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = ".".join(k.split(".")[start_idx:])
            new_state_dict[name] = v
        return new_state_dict
    state_dict = copy_state_dict(model, torch.load(opt.continue_model))
    model.load_state_dict(state_dict)

model = torch.nn.DataParallel(model).cuda()

# eğitilecek olan ağırlıkları filtreleyelim ve toplam parametre sayısını bulalım
filtered_parameters = []
params_num = []
for name, param in filter(lambda name_param: name_param[1].requires_grad, model.named_parameters()):
    filtered_parameters.append(param)
    params_num.append(np.prod(param.size()))
print('Eğitilebilir parametre sayısı : ', sum(params_num))
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR

In [ ]:
model.train()
criterion = torch.nn.CrossEntropyLoss(ignore_index=0).cuda()  # kayıp fonksiyonu
loss_avg = Averager()  # modelin eğitimi boyunca hesaplanan kayıp değerinin ortalamasınu hesaplayama

In [ ]:
# optimizasyon metodunu oluşturma
if opt.optimizer == 'Adam':
    optimizer = optim.Adam(filtered_parameters, lr=opt.lr, betas=(opt.beta1, 0.999))
elif opt.optimizer == 'SGD':
    optimizer = optim.SGD(filtered_parameters, lr=opt.lr,
                            momentum=0.9, weight_decay=5e-4, nesterov=True)
else:
    optimizer = optim.Adadelta(filtered_parameters, lr=opt.lr, rho=opt.rho, eps=opt.eps)
print("Optimizer:")
print(optimizer)

In [ ]:
# modelin eğitimi
torch.cuda.empty_cache()
start_iter = 0
start_time = time.time()
best_accuracy = -1
best_norm_ED = 1e+6
i = start_iter

while(True):
    if i == opt.num_iter:
        break
    for p in model.parameters():
        p.requires_grad = True

    image_tensors, labels = train_dataset.get_batch()  # sonraki verilerin (batch) okunması
    image  = image_tensors.cuda()
    text, length = converter.encode(labels)  # gerçek referans metindeki karakterlerin sayılara çevrilmesi
    batch_size = image.size(0)
    
    # image -> modele girdi olarak verilen metin resmi
    # text -> resimdeki metnin gerçek referans değeri
    preds = model(image, text)
    target = text[:, 1:]  # without [GO] Symbol
    cost = criterion(preds.view(-1, preds.shape[-1]), target.contiguous().view(-1))

    model.zero_grad()
    cost.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)  # gradient clipping with 5 (Default)
    optimizer.step()

    loss_avg.add(cost)

    if (i+1) % opt.valInterval == 0:
        torch.save(model.state_dict(), f'./saved_models/model_iterasyon{i+1}.pth')
    print('iteration: {}, Ortalama Loss Değeri: {}'.format(i+1, loss_avg.sum / loss_avg.n_count))
    i += 1

## Optik Karakter Tanıma: Metin Tespiti ile Metin Tanıma'nın Birleştirilmesi

In [ ]:
torch.cuda.empty_cache()
# ICDAR_2015 veri setinden herhangi bir resmi yükleyelim
resim_dosyasi = 'ICDAR_2015/test_img/test3.jpg'
resim = Image.open(resim_dosyasi)
# kullanacağımız model dosyalarının konumları
metin_tespiti_model_dosyasi = 'EAST/pths/east_vgg16.pth'
# metin_tanima_model_dosyasi = 'deep_text_recognition_benchmark/pths/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth'
metin_tanima_model_dosyasi = '/content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR/deep_text_recognition_benchmark/pths/TPS-ResNet-BiLSTM-Attn-case-sensitive.pth'

In [ ]:
# MODELLERİN OLUŞTURULMASI
# metin tespiti modeli
model_detection = EASTModel(pretrained=False).to("cuda:0")
model_detection.load_state_dict(torch.load(metin_tespiti_model_dosyasi))
# modelin test moduna alınması
model_detection.eval()

# metin tanıma modeli
# tahminin maksimum karakter sayısı ve metnin tahmininin tutulacağı değişkenler
tahminin_uzunlugu = torch.cuda.IntTensor([opt.batch_max_length] * 1)
tahminin_metni = torch.cuda.LongTensor(1, opt.batch_max_length + 1).fill_(0)

# modelin tahminlerini metne çeviren modülün oluşturulması
converter = AttnLabelConverter(opt.character)
# modelin oluşturulması ve önceden eğitilmiş modelin yüklenmesi
model_recognition = Model(opt)
# torch.nn.DataParallel ile eğitilen modeller yine bu şekilde çağrılmalıdır
model_recognition = torch.nn.DataParallel(model_recognition).cuda()
model_recognition.load_state_dict(torch.load(metin_tanima_model_dosyasi))

In [ ]:
def ocr(resim, model_detection, model_recognition):
    # METİN TESPİTİ
    # resmi metin tespiti modeline girdi olarak verebilmek için pytorch tensörune dönüştürme
    resim_32, boy_orani, en_orani = detect.resize_img(resim)
    resim_tensor = detect.load_pil(resim_32)
    resim_tensor = resim_tensor.to("cuda:0")
    with torch.no_grad():
        skor, geometri = model_detection(resim_tensor)

    tahmin_dortgenler = detect.get_boxes(skor.squeeze(0).cpu().numpy(), geometri.squeeze(0).cpu().numpy())
    tahmin_dortgenler = detect.adjust_ratio(tahmin_dortgenler, en_orani, boy_orani)
    print("Tahmin edilen metin sayısı", tahmin_dortgenler.shape[0])
    
    # METİN TANIMA
    tensor_donusturucu = ResizeNormalize((opt.imgW, opt.imgH))
    tahmin_metinler = []
    for dortgen in tahmin_dortgenler:
        dortgen = dortgen[:8].reshape((4, 2))
        print(dortgen)
        x_sol = np.min(dortgen[:, 0])
        x_sag = np.max(dortgen[:, 0])
        y_ust = np.min(dortgen[:, 1])
        y_alt = np.max(dortgen[:, 1])
        # tahmin edilen metnin olduğu bölgenin resimden kesilmesi
        metin_parcasi = resim.crop((x_sol, y_ust, x_sag, y_alt)).convert('L')
        resim_tensor = tensor_donusturucu(metin_parcasi.copy()).cuda()
        resim_tensor = resim_tensor.view(1, *resim_tensor.size())
        tahminin_uzunlugu = torch.cuda.IntTensor([opt.batch_max_length])
        tahminin_metni = torch.cuda.LongTensor(1, opt.batch_max_length + 1).fill_(0)

        # tahminin yapılması
        tahmin = model_recognition(resim_tensor, tahminin_metni, is_train=False)

        # en yüksek ihtimalle tahmin edilen karakterlerin indekslerini çıkarır
        _, tahmin_indeksi = tahmin.max(2)
        # buradaki sıfır resim sayımız sadece 1 olduğundan ilk elemanı almak için kullanılıyor
        tahmin_metni = converter.decode(tahmin_indeksi, tahminin_uzunlugu)[0]
        # "bitiş" özel karakterinin "([s])" çıkarılması
        tahmin_metni = tahmin_metni[:tahmin_metni.find('[s]')]
        # tahmin edilen metnin resim ile gösterilmesi
        tahmin_metinler.append(tahmin_metni)
    return tahmin_dortgenler, tahmin_metinler

In [ ]:
tahmin_dortgenler, tahmin_metinler = ocr(resim, model_detection, model_recognition)
print(tahmin_metinler)

In [ ]:
# tespit edilen tüm sonuçları gösterelim
plt.imshow(resim)
for dortgen, metin in zip(tahmin_dortgenler, tahmin_metinler):
    x1, y1, x2, y2, x3, y3, x4, y4, egim = dortgen
    plt.plot((x1, x2, x3, x4, x1), (y1, y2, y3, y4, y1), marker='o', color='green', linewidth=2)
    plt.text(x2 + 10, y2 + 5, metin, color='red',\
             bbox=dict(facecolor='white', alpha=0.5), fontsize=15)
plt.show()

## Modellerin Performans Ölçümü

### Metin Tespiti Modeli Performans Ölçümü
Metin Tespiti modelinin performans ölçümünü Kesinlik (Precision), Duyarlılık (Recall) ve F1-Skoru (F1-Score) ile ölçelim

In [ ]:
%cd EAST
torch.cuda.empty_cache()
test_img_path = '../ICDAR_2015/test_img'
submit_path = './submit'
if os.path.exists(submit_path):
    shutil.rmtree(submit_path) 
os.mkdir(submit_path)

start_time = time.time()
detect.detect_dataset(model_detection, device, test_img_path, submit_path)
os.chdir(submit_path)
res = subprocess.getoutput('zip -q submit.zip *.txt')
res = subprocess.getoutput('mv submit.zip ../')
os.chdir('../')
res = subprocess.getoutput('python ./evaluate/script.py –g=./evaluate/gt.zip –s=./submit.zip')
print(res)
os.remove('./submit.zip')
print('eval time is {}'.format(time.time()-start_time))	

shutil.rmtree(submit_path)
%cd ..

In [ ]:
%cd /content/gdrive/MyDrive/Calisma_ortami/Uygulamalar/Optik_Karakter_Tanima/OCR

### Metin Tanıma Modeli Performans Ölçümü

Metin Tanıma modelinin performans ölçümünü Doğruluk (Accuracy) ile ölçelim

In [ ]:
torch.cuda.empty_cache()
metin_resim_isimleri = os.listdir('ICDAR_2015_metinler/test_img')
metin_resimleri_konumları = [os.path.join('ICDAR_2015_metinler/test_img',\
                             resim_ismi) for resim_ismi in metin_resim_isimleri]
tensor_donusturucu = ResizeNormalize((opt.imgW, opt.imgH))
referans_metinler = {}
with open('ICDAR_2015_metinler/test_img_gt.txt', 'r') as f_r:
    for satir in f_r:
        resim_ismi, metin = satir.strip('\n').split('\t')
        resim_ismi = resim_ismi.split('/')[-1]
        referans_metinler[resim_ismi] = metin
toplam_dogru = 0
print('', end='')
for sayi, metin_resim_konumu in enumerate(metin_resimleri_konumları):
    metin_resmi = Image.open(metin_resim_konumu).convert('L')
    resim_tensor = tensor_donusturucu(metin_resmi).cuda()
    resim_tensor = resim_tensor.view(1, *resim_tensor.size())
    tahminin_uzunlugu = torch.cuda.IntTensor([opt.batch_max_length])
    tahminin_metni = torch.cuda.LongTensor(1, opt.batch_max_length + 1).fill_(0)

    # tahminin yapılması
    tahmin = model_recognition(resim_tensor, tahminin_metni, is_train=False)

    # en yüksek ihtimalle tahmin edilen karakterlerin indekslerini çıkarır
    _, tahmin_indeksi = tahmin.max(2)
    # buradaki sıfır resim sayımız sadece 1 olduğundan ilk elemanı almak için kullanılıyor
    tahmin_metni = converter.decode(tahmin_indeksi, tahminin_uzunlugu)[0]
    tahmin_metni = tahmin_metni[:tahmin_metni.find('[s]')]
    if (referans_metinler[metin_resim_konumu.split('/')[-1]] == tahmin_metni):
        toplam_dogru += 1
    print('\rresim {}/{}, {}'.format(sayi + 1, len(metin_resimleri_konumları), toplam_dogru), end='')
print('\nDoğruluk (Acurracy):', toplam_dogru / len(metin_resimleri_konumları))

## Optik Karakter Tanıma Modellerinin Kendi Veriniz Üzerinde Eğitilip Uygulanması

Eğer kendi veriniz üzerinde Optik Karakter Tanıma uygulamak isterseniz;
* Öncelikle burada kullanmış olduğumuz, önceden eğitilmiş modelleri yazdığımız ocr fonksiyonuyla veriniz üzerinde uygulayabilirsiniz
* Performansın daha iyi olmasını isterseniz, bu modelleri temel model olarak kullanarak kendi veriniz üzerinde sağladığımız eğitim kodlarıyla eğitime devam edebilirsiniz (fin2-tuning). Bunun için verinizi ICDAR_2015 veri setiyle aynı formatta hazırlayıp, resimler için işaretlemeleri yapmalısınız. ocr fonksiyonunun kulllanacağı modelleri, kendi veriniz üzerinde eğitilmiş modeller ile değiştirmelisiniz.